##토큰화와 벡터화 같은 기본적인 NLP 전처리를 마친 후 pycaret 적용



1.   BOW-TF (Countvectorizer)
2.   BOW-TFIDF (TFIDFvectorizer) ☑





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


pycaret 설치 오류 관련 링크 (버전 문제) <br/>
https://coding-potato.tistory.com/7

pycaret 동작 방법 <br/>
https://www.notion.so/pycaret-836baab165694d519ad90fd8c1dc6713?pvs=4

In [ ]:
#pycaret 돌릴 때 필요한 코드, 단 처음 돌린 후 런타임 재시작하고 다시 돌려야 돌아가는 단점 존재
!pip install -U pypandoc
!pip install  pycaret[full]
#!pip install markupsafe==2.1.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip show packaging

Name: packaging
Version: 21.3
Summary: Core utilities for Python packages
Home-page: https://github.com/pypa/packaging
Author: Donald Stufft and individual contributors
Author-email: donald@stufft.io
License: BSD-2-Clause or Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: pyparsing
Required-by: arviz, astropy, bokeh, build, dask, db-dtypes, deprecation, distributed, docker, fastai, fastcore, geopandas, google-cloud-bigquery, gradio_client, h5netcdf, holoviews, huggingface-hub, jupyter-server, matplotlib, mlflow, nbconvert, optuna, pooch, pytest, qtconsole, QtPy, ray, scikit-image, shap, sktime, spacy, Sphinx, statsmodels, tables, tensorboardX, tensorflow, thinc, xarray


In [ ]:
pip install "packaging<=21.3"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from packaging.version import Version

version_string = "21.3"
version = Version(version_string)

print(version)

21.3




*   하단의 코드에서 패키징 버전이 23.1이 아닌 **21.3 이하 버전**일 때 pycaret import 가능

In [ ]:
import packaging

print(packaging.__version__)

21.3


In [ ]:
from pycaret.classification import *

In [ ]:
import re
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
train_r = '/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/train.csv'
test_r= '/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/test.csv'
temp_r='/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/train_tfidf_m.csv'
tempT_r='/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/test_tfidf_m.csv'

In [ ]:
train = pd.read_csv(train_r)
test = pd.read_csv(test_r)
new = pd.read_csv(temp_r)
newT=pd.read_csv(tempT_r)

In [ ]:
new

,Unnamed: 0,ID,first_party,second_party,first_party_winner,abuse,access,according,account,act,...,witness,woman,work,worked,worker,working,would,writ,year,york
0,0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,1,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
1,1,TRAIN_0001,Stephen Duncan,Lawrence Owens,0,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.281085,0.089681,0.0
2,2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",1,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
3,3,TRAIN_0003,Linkletter,Walker,0,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
4,4,TRAIN_0004,William Earl Fikes,Alabama,1,0.0,0.18325,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.103997,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,2473,TRAIN_2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",1,0.0,0.00000,0.000000,0.0,0.157253,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.103833,0.000000,0.213030,0.0
2474,2474,TRAIN_2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.",1,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.103559,0.000000,0.000000,0.0
2475,2475,TRAIN_2475,Peguero,United States,0,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
2476,2476,TRAIN_2476,Immigration and Naturalization Service,St. Cyr,0,0.0,0.00000,0.000000,0.0,0.174673,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0


In [ ]:
newT

,Unnamed: 0,ID,first_party,second_party,abuse,access,according,account,act,acting,...,witness,woman,work,worked,worker,working,would,writ,year,york
0,0,TEST_0000,Salerno,United States,0.0,0.0,0.000000,0.0,0.136351,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,1,TEST_0001,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc.",0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,2,TEST_0002,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al.",0.0,0.0,0.000000,0.0,0.069597,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,3,TEST_0003,Harold Kaufman,United States,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,4,TEST_0004,Berger,Hanlon,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,1235,TEST_1235,"Haitian Centers Council, Inc., et al.","Chris Sale, Acting Commissioner, Immigration A...",0.0,0.0,0.198955,0.0,0.085500,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166307
1236,1236,TEST_1236,Whitman,"American Trucking Associations, Inc.",0.0,0.0,0.000000,0.0,0.067014,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1237,1237,TEST_1237,Linda A. Matteo and John J. Madigan,William G. Barr,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1238,1238,TEST_1238,Washington State Apple Advertising Commission,Hunt,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [ ]:
new.drop(columns=['Unnamed: 0','first_party','second_party'],inplace=True)
newT.drop(columns=['Unnamed: 0','first_party','second_party'],inplace=True)
new

,ID,first_party_winner,abuse,access,according,account,act,acting,action,activity,...,witness,woman,work,worked,worker,working,would,writ,year,york
0,TRAIN_0000,1,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.14127,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
1,TRAIN_0001,0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.281085,0.089681,0.0
2,TRAIN_0002,1,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
3,TRAIN_0003,0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
4,TRAIN_0004,1,0.0,0.18325,0.000000,0.0,0.000000,0.0,0.000000,0.00000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.103997,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,TRAIN_2473,1,0.0,0.00000,0.000000,0.0,0.157253,0.0,0.000000,0.00000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.103833,0.000000,0.213030,0.0
2474,TRAIN_2474,1,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.109291,0.00000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.103559,0.000000,0.000000,0.0
2475,TRAIN_2475,0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
2476,TRAIN_2476,0,0.0,0.00000,0.000000,0.0,0.174673,0.0,0.000000,0.00000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0


In [ ]:
new=pd.merge(new,train)
newT=pd.merge(newT,test)

In [ ]:
new

,ID,first_party_winner,abuse,access,according,account,act,acting,action,activity,...,worked,worker,working,would,writ,year,york,first_party,second_party,facts
0,TRAIN_0000,1,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.14127,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ..."
1,TRAIN_0001,0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.000000,0.281085,0.089681,0.0,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...
2,TRAIN_0002,1,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...
3,TRAIN_0003,0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,Linkletter,Walker,Victor Linkletter was convicted in state court...
4,TRAIN_0004,1,0.0,0.18325,0.000000,0.0,0.000000,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.103997,0.000000,0.000000,0.0,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,TRAIN_2473,1,0.0,0.00000,0.000000,0.0,0.157253,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.103833,0.000000,0.213030,0.0,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",Congress amended the Clean Air Act through the...
2474,TRAIN_2474,1,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.109291,0.00000,...,0.0,0.0,0.0,0.103559,0.000000,0.000000,0.0,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.","Alliance Bond Fund, Inc., an investment fund, ..."
2475,TRAIN_2475,0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,Peguero,United States,"In 1992, the District Court sentenced Manuel D..."
2476,TRAIN_2476,0,0.0,0.00000,0.000000,0.0,0.174673,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per..."


In [ ]:
new.set_index('ID', drop = True, inplace=True)
newT.set_index('ID', drop = True, inplace=True)

In [ ]:
new.drop(columns=['facts'],inplace=True)
newT.drop(columns=['facts'],inplace=True)

In [ ]:
new

,first_party_winner,abuse,access,according,account,act,acting,action,activity,actual,...,work,worked,worker,working,would,writ,year,york,first_party,second_party
ID,,,,,,,,,,,,,,,,,,,,,
TRAIN_0000,1,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.14127,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,Phil A. St. Amant,Herman A. Thompson
TRAIN_0001,0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.281085,0.089681,0.0,Stephen Duncan,Lawrence Owens
TRAIN_0002,1,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,Billy Joe Magwood,"Tony Patterson, Warden, et al."
TRAIN_0003,0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,Linkletter,Walker
TRAIN_0004,1,0.0,0.18325,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.103997,0.000000,0.000000,0.0,William Earl Fikes,Alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRAIN_2473,1,0.0,0.00000,0.000000,0.0,0.157253,0.0,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.103833,0.000000,0.213030,0.0,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al."
TRAIN_2474,1,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.109291,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.103559,0.000000,0.000000,0.0,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc."
TRAIN_2475,0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,Peguero,United States


In [ ]:
newT

,abuse,access,according,account,act,acting,action,activity,actual,additional,...,work,worked,worker,working,would,writ,year,york,first_party,second_party
ID,,,,,,,,,,,,,,,,,,,,,
TEST_0000,0.0,0.0,0.000000,0.0,0.136351,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,Salerno,United States
TEST_0001,0.0,0.0,0.000000,0.0,0.000000,0.0,0.261991,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc."
TEST_0002,0.0,0.0,0.000000,0.0,0.069597,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al."
TEST_0003,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,Harold Kaufman,United States
TEST_0004,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,Berger,Hanlon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TEST_1235,0.0,0.0,0.198955,0.0,0.085500,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166307,"Haitian Centers Council, Inc., et al.","Chris Sale, Acting Commissioner, Immigration A..."
TEST_1236,0.0,0.0,0.000000,0.0,0.067014,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,Whitman,"American Trucking Associations, Inc."
TEST_1237,0.0,0.0,0.000000,0.0,0.000000,0.0,0.069865,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,Linda A. Matteo and John J. Madigan,William G. Barr


In [ ]:
#fix_imbalance=True를 통해 SMOTE를 적용하여 클래스가 적은 데이터의 표본을 임의 값으로 변형하여 데이터 추가 적용
exp=setup(data=new, target='first_party_winner', session_id=4454, fix_imbalance=True)

,Description,Value
0,Session id,4454
1,Target,first_party_winner
2,Target type,Binary
3,Original data shape,"(2478, 673)"
4,Transformed data shape,"(3052, 673)"
5,Transformed train set shape,"(2308, 673)"
6,Transformed test set shape,"(744, 673)"
7,Numeric features,670
8,Categorical features,2
9,Preprocess,True


In [ ]:
 best_15 = compare_models(sort='Accuracy', n_select=15)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
qda,Quadratic Discriminant Analysis,0.6655,0.5000,1.0000,0.6655,0.7992,0.0000,0.0000,1.0080
et,Extra Trees Classifier,0.6568,0.5512,0.9306,0.6759,0.7830,0.0525,0.0736,1.7860
lr,Logistic Regression,0.6551,0.5567,0.8813,0.6883,0.7728,0.0999,0.1132,1.7500
xgboost,Extreme Gradient Boosting,0.6505,0.5323,0.9193,0.6743,0.7778,0.0417,0.0535,3.0120
lightgbm,Light Gradient Boosting Machine,0.6459,0.5146,0.9254,0.6691,0.7765,0.0190,0.0344,1.8420
gbc,Gradient Boosting Classifier,0.6447,0.5552,0.8778,0.6810,0.7665,0.0678,0.0750,4.3150
ridge,Ridge Classifier,0.6442,0.0000,0.8709,0.6824,0.7651,0.0738,0.0835,0.6310
catboost,CatBoost Classifier,0.6430,0.5471,0.8916,0.6762,0.7684,0.0453,0.0482,69.6890
svm,SVM - Linear Kernel,0.6264,0.0000,0.8356,0.6787,0.7451,0.0509,0.0593,0.9760
rf,Random Forest Classifier,0.6187,0.5470,0.7961,0.6837,0.7325,0.0668,0.0711,1.7990


Processing:   0%|          | 0/83 [00:00<?, ?it/s]

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/sample_submission.csv')
submit['first_party_winner'] = y_pred
submit.to_csv('./proces_tf_lg.csv', index=False)
print('Done')

Done



### ✔ Quadratic Discriminant Analysis
*   **Predict '0' Result** </br>
train : 580 </br>
test  : 0 </br>

In [ ]:
tuned_model = tune_model(best_15[0], choose_better=True, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5632,0.6052,0.5517,0.7273,0.6275,0.1231,0.1301
1,0.5287,0.6027,0.4655,0.7297,0.5684,0.1022,0.1151
2,0.4885,0.5412,0.4052,0.7015,0.5137,0.0498,0.0585
3,0.5517,0.6189,0.4828,0.7568,0.5895,0.1460,0.1644
4,0.5318,0.5723,0.4957,0.7125,0.5846,0.0862,0.0938
5,0.4913,0.5270,0.5043,0.6517,0.5686,-0.0271,-0.0285
6,0.5607,0.6022,0.5304,0.7349,0.6162,0.1329,0.1428
7,0.5145,0.5403,0.5217,0.6742,0.5882,0.0196,0.0205
8,0.5434,0.6306,0.4522,0.7647,0.5683,0.1468,0.1704


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Quadratic Discriminant Analysis,0.8995,0.8498,1.0000,0.8688,0.9298,0.7561,0.7796


In [ ]:
y_pred.value_counts()

1    1898
0     580
Name: prediction_label, dtype: int64

In [ ]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [ ]:
y_pred.value_counts()

1    1240
Name: prediction_label, dtype: int64


### ✔ Extra Trees Classifier
*   **Predict 0 Result** </br>
train : 692 </br>
test  : 121 </br>


In [ ]:
%time tuned_model = tune_model(best_15[1],choose_better=True, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6437,0.5162,0.8707,0.6824,0.7652,0.0700,0.0798
1,0.6667,0.5627,0.8879,0.6959,0.7803,0.1300,0.1482
2,0.6609,0.5199,0.9138,0.6839,0.7823,0.0829,0.1042
3,0.6897,0.5743,0.9138,0.7067,0.7970,0.1818,0.2121
4,0.6474,0.5130,0.9043,0.6753,0.7732,0.0507,0.0638
5,0.6416,0.5076,0.9043,0.6710,0.7704,0.0302,0.0387
6,0.6936,0.5955,0.9043,0.7123,0.7969,0.2077,0.2344
7,0.6763,0.5407,0.9304,0.6903,0.7926,0.1241,0.1590
8,0.6705,0.5509,0.8957,0.6959,0.7833,0.1395,0.1608


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


CPU times: user 7.74 s, sys: 2.9 s, total: 10.6 s
Wall time: 2min 57s


In [ ]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.8358,0.8494,0.9181,0.8477,0.8815,0.6153,0.6205


In [ ]:
y_pred.value_counts()

1    1786
0     692
Name: prediction_label, dtype: int64

In [ ]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [ ]:
y_pred.value_counts()

1    1119
0     121
Name: prediction_label, dtype: int64


### ✔ Logistic Regression
*   **Predict 0 Result** </br>
train : 688 </br>
test  : 166 </br>


In [ ]:
%time tuned_model = tune_model(best_15[2],choose_better=True, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6379,0.5513,0.8448,0.6853,0.7568,0.0780,0.0850
1,0.6897,0.5675,0.8966,0.7123,0.7939,0.1980,0.2212
2,0.6207,0.5163,0.8534,0.6689,0.7500,0.0100,0.0114
3,0.6724,0.6103,0.8966,0.6980,0.7849,0.1407,0.1622
4,0.6358,0.5457,0.8783,0.6733,0.7623,0.0393,0.0465
5,0.6243,0.4516,0.8609,0.6689,0.7529,0.0187,0.0215
6,0.7052,0.5783,0.9217,0.7162,0.8061,0.2300,0.2653
7,0.6647,0.5303,0.8783,0.6966,0.7769,0.1372,0.1533
8,0.6647,0.5741,0.8696,0.6993,0.7752,0.1456,0.1598


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
predictions = predict_model(best_15[2], data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.8818,0.9407,0.9539,0.8788,0.9148,0.7227,0.7292


In [ ]:
y_pred.value_counts()

1    1790
0     688
Name: prediction_label, dtype: int64

In [ ]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [ ]:
y_pred.value_counts()

1    1074
0     166
Name: prediction_label, dtype: int64


### ✔ XGBoost
*   **Predict 0 Result** </br>
train : 597 </br>
test  : 57 </br>

In [ ]:
%time tuned_model = tune_model(best_15[3],choose_better=True, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6609,0.5131,0.9828,0.6667,0.7944,0.0000,0.0000
1,0.6609,0.5970,0.9741,0.6686,0.7930,0.0112,0.0243
2,0.6437,0.5392,0.9655,0.6588,0.7832,-0.0449,-0.1085
3,0.6782,0.6373,0.9914,0.6765,0.8042,0.0562,0.1356
4,0.6647,0.5119,0.9739,0.6707,0.7943,0.0330,0.0661
5,0.6532,0.5483,0.9565,0.6667,0.7857,0.0105,0.0185
6,0.6705,0.5803,0.9739,0.6747,0.7972,0.0548,0.1027
7,0.6358,0.4965,0.9304,0.6605,0.7726,-0.0223,-0.0345
8,0.6474,0.5378,0.9652,0.6607,0.7845,-0.0227,-0.0494


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


CPU times: user 26.9 s, sys: 4.46 s, total: 31.3 s
Wall time: 7min 3s


In [ ]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.8927,0.9390,0.9897,0.8676,0.9246,0.7409,0.7606


In [ ]:
y_pred.value_counts()

1    1881
0     597
Name: prediction_label, dtype: int64

In [ ]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [ ]:
y_pred.value_counts()

1    1183
0      57
Name: prediction_label, dtype: int64


### ✔ LGBM
*   **Predict 0 Result** </br>
train : 645 </br>
test  : 131 </br>


In [ ]:
tuned_model = tune_model(best_15[4])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6552,0.5326,0.9138,0.6795,0.7794,0.0625,0.0801
1,0.6782,0.6684,0.9397,0.6899,0.7956,0.1158,0.1547
2,0.6494,0.5220,0.9483,0.6667,0.7829,0.0000,0.0000
3,0.6609,0.6171,0.9224,0.6815,0.7839,0.0733,0.0958
4,0.6301,0.5198,0.8696,0.6711,0.7576,0.0289,0.0338
5,0.6590,0.4922,0.9217,0.6795,0.7823,0.0723,0.0946
6,0.6416,0.5642,0.8870,0.6755,0.7669,0.0498,0.0597
7,0.6358,0.5739,0.9304,0.6605,0.7726,-0.0223,-0.0345
8,0.6705,0.5658,0.8783,0.7014,0.7799,0.1562,0.1730


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8951,0.9570,0.9770,0.8789,0.9253,0.7506,0.7626


In [ ]:
y_pred.value_counts()

1    1833
0     645
Name: prediction_label, dtype: int64

In [ ]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [ ]:
y_pred.value_counts()

1    1109
0     131
Name: prediction_label, dtype: int64


### ✔ XGBoost
*   **Predict 0 Result** </br>
train : 597 </br>
test  : 57 </br>

In [ ]:
%time tuned_model = tune_model(best_15[3],choose_better=True, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6609,0.5131,0.9828,0.6667,0.7944,0.0000,0.0000
1,0.6609,0.5970,0.9741,0.6686,0.7930,0.0112,0.0243
2,0.6437,0.5392,0.9655,0.6588,0.7832,-0.0449,-0.1085
3,0.6782,0.6373,0.9914,0.6765,0.8042,0.0562,0.1356
4,0.6647,0.5119,0.9739,0.6707,0.7943,0.0330,0.0661
5,0.6532,0.5483,0.9565,0.6667,0.7857,0.0105,0.0185
6,0.6705,0.5803,0.9739,0.6747,0.7972,0.0548,0.1027
7,0.6358,0.4965,0.9304,0.6605,0.7726,-0.0223,-0.0345
8,0.6474,0.5378,0.9652,0.6607,0.7845,-0.0227,-0.0494


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


CPU times: user 10.8 s, sys: 5.95 s, total: 16.7 s
Wall time: 1min 22s


In [ ]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.8927,0.9390,0.9897,0.8676,0.9246,0.7409,0.7606


In [ ]:
y_pred.value_counts()

1    1881
0     597
Name: prediction_label, dtype: int64

In [ ]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [ ]:
y_pred.value_counts()

1    1183
0      57
Name: prediction_label, dtype: int64


### ✔ Naive Bayes
*   **Predict 0 Result** </br>
train : 843 </br>
test  : 383 </br>

In [ ]:
%time tuned_model = tune_model(best_15[13],choose_better=True, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5517,0.5123,0.6810,0.6583,0.6695,-0.0263,-0.0264
1,0.5517,0.5001,0.6897,0.6557,0.6723,-0.0354,-0.0355
2,0.5460,0.4804,0.6552,0.6609,0.6580,-0.0172,-0.0172
3,0.5690,0.5349,0.6466,0.6881,0.6667,0.0586,0.0588
4,0.5434,0.5106,0.6348,0.6636,0.6489,-0.0031,-0.0031
5,0.5954,0.5296,0.6870,0.6991,0.6930,0.0999,0.0999
6,0.6185,0.5670,0.7130,0.7130,0.7130,0.1441,0.1441
7,0.5665,0.5250,0.6522,0.6818,0.6667,0.0477,0.0478
8,0.6301,0.6043,0.7565,0.7073,0.7311,0.1406,0.1415


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
CPU times: user 11.4 s, sys: 6.24 s, total: 17.6 s
Wall time: 1min 46s


In [ ]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.7708,0.7945,0.8235,0.8306,0.8270,0.4873,0.4874


In [ ]:
y_pred.value_counts()

1    1635
0     843
Name: prediction_label, dtype: int64

In [ ]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [ ]:
y_pred.value_counts()

1    857
0    383
Name: prediction_label, dtype: int64

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/sample_submission.csv')
submit['first_party_winner'] = y_pred
submit.to_csv('./proces_tfidf_naive.csv', index=False)
print('Done')

Done



### ✔ CatBoost
*   **Predict 0 Result** </br>
train : 1126 </br>
test  : 540 </br>


In [ ]:
%time tuned_model = tune_model(best_15[7],choose_better=True, optimize='Accuracy')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,10:53:56
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,CatBoost Classifier


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

In [ ]:
y_pred.value_counts()

In [ ]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [ ]:
y_pred.value_counts()

In [ ]:
#blended_l = blend_models(estimator_list = best_5, fold = 5, optimize = 'Accuracy')
#pred_holdout = predict_model(blended_l)
#final_model_l = finalize_model(blended_l)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6657,0.5494,1.0000,0.6657,0.7993,0.0000,0.0000
1,0.6657,0.5842,1.0000,0.6657,0.7993,0.0000,0.0000
2,0.6657,0.5679,1.0000,0.6657,0.7993,0.0000,0.0000
3,0.6657,0.5860,1.0000,0.6657,0.7993,0.0000,0.0000
4,0.6647,0.6283,1.0000,0.6647,0.7986,0.0000,0.0000
Mean,0.6655,0.5832,1.0000,0.6655,0.7992,0.0000,0.0000
Std,0.0004,0.0262,0.0000,0.0004,0.0003,0.0000,0.0000


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.6653,0.5416,1.0000,0.6653,0.7990,0.0000,0.0000
